In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('kmcalong').getOrCreate()

In [0]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.ml.clustering import KMeans

In [0]:
df = sqlContext.sql("SELECT * FROM seeds_dataset_csv")
df.describe().show()

+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
summary| area| perimeter| compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient| length_of_groove|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
 count| 210| 210| 210| 210| 210| 210| 210|
 mean|14.847523809523816|14.559285714285718| 0.8709985714285714| 5.628533333333335| 3.258604761904762| 3.7001999999999997| 5.408071428571429|
 stddev|2.9096994306873647|1.3059587265640225|0.023629416583846364|0.44306347772644983|0.3777144449065867| 1.5035589702547392|0.49148049910240543|
 min| 10.59| 12.41| 0.8081| 4.899| 2.63| 0.765| 4.519|
 max| 21.18| 17.25| 0.9183| 6.675| 4.033| 8.456| 6.55|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+

In [0]:
df.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)

In [0]:
df.head(1)

Out[6]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [0]:
df.columns

Out[7]: ['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [0]:
ass = VectorAssembler(inputCols=df.columns,
                     outputCol='features')

In [0]:
fdf=ass.transform(df)
fdf.printSchema()


root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
scaler = StandardScaler(inputCol='features',outputCol='scaledFeatures',)

In [0]:
scaler_model=scaler.fit(fdf)

In [0]:
ffdf = scaler_model.transform(fdf)

In [0]:
ffdf.head(1)

Out[20]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [0]:
km = KMeans(featuresCol='scaledFeatures',k=3)

In [0]:
model = km.fit(ffdf)

In [0]:
wssse = model.summary.trainingCost
wssse

Out[23]: 428.6720009438313

In [0]:
model.clusterCenters()

Out[24]: [array([ 4.93382436, 10.94691274, 37.30542404, 12.41332714, 8.60366812,
 1.82917353, 10.40106154]),
 array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446, 9.7892399 ,
 2.41585013, 12.29286107]),
 array([ 4.06660859, 10.14191893, 35.84098009, 11.81592066, 7.52397236,
 3.1823335 , 10.39801233])]

In [0]:
model.transform(ffdf).select('prediction').show()

+----------+
prediction|
+----------+
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 1|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 2|
+----------+
only showing top 20 rows